In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
!pip install --upgrade google-cloud


In [ ]:
import os
os.environ["GOOGLE_CLOUD_PROJECT"] = "astral-net-472017-i3"


In [ ]:
!gcloud config set project astral-net-472017-i3


Updated property [core/project].


In [ ]:
!gcloud config get-value project


astral-net-472017-i3


In [ ]:
!gcloud services enable compute.googleapis.com
!gcloud services enable storage.googleapis.com


In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification, pipeline

# Load processor & model
processor = AutoImageProcessor.from_pretrained("prithivMLmods/Deep-Fake-Detector-v2-Model")
model = AutoModelForImageClassification.from_pretrained("prithivMLmods/Deep-Fake-Detector-v2-Model")

# Create pipeline (use image_processor instead of tokenizer)
pipe = pipeline("image-classification", model=model, image_processor=processor)

# Test
result = pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/parrots.png")
print(result)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


[{'label': 'Deepfake', 'score': 0.7101955413818359}, {'label': 'Realism', 'score': 0.28980448842048645}]


In [ ]:
model.save_pretrained("deepfake_model")
processor.save_pretrained("deepfake_model")


['deepfake_model/preprocessor_config.json']

In [ ]:
!gsutil cp -r deepfake_model gs://hackathon1512/deepfake_model


Copying file://deepfake_model/preprocessor_config.json [Content-Type=application/json]...
Copying file://deepfake_model/config.json [Content-Type=application/json]...
Copying file://deepfake_model/model.safetensors [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
Operation completed over 3 objects/327.3 MiB.                   

In [ ]:
!gcloud services enable aiplatform.googleapis.com


In [ ]:
!pip install google-cloud-aiplatform --upgrade


In [ ]:
from google.cloud import aiplatform

# Replace with your project and region
PROJECT_ID = "astral-net-472017-i3"   # your GCP project ID
REGION = "asia-south1"               # or asia-south1 for India
BUCKET = "hackathon1512"          # the GCS bucket you uploaded deepfake_model

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET)


In [ ]:
model.save_pretrained("deepfake_model")
processor.save_pretrained("deepfake_model")


['deepfake_model/preprocessor_config.json']

In [ ]:
!gsutil cp -r deepfake_model gs://hackathon1512/deepfake_model


Copying file://deepfake_model/preprocessor_config.json [Content-Type=application/json]...
Copying file://deepfake_model/config.json [Content-Type=application/json]...
Copying file://deepfake_model/model.safetensors [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
Operation completed over 3 objects/327.3 MiB.                   

In [ ]:
!pip install torch-model-archiver torchserve


In [ ]:
%%writefile deepfake_handler.py
import torch
from ts.torch_handler.vision_handler import VisionHandler

class DeepFakeHandler(VisionHandler):
    def __init__(self):
        super(DeepFakeHandler, self).__init__()


Writing deepfake_handler.py


In [ ]:
# ============================================
# CELL 1: Authentication and Setup
# ============================================
from google.colab import auth
import os
auth.authenticate_user()

# ============================================
# CELL 2: Install Dependencies
# ============================================
!pip install --upgrade google-cloud-storage google-cloud-aiplatform google-cloud-functions
!pip install transformers torch torchvision pillow opencv-python-headless
!pip install flask flask-cors requests numpy pandas
!pip install torch-model-archiver torchserve

# ============================================
# CELL 3: Project Configuration
# ============================================
import os
from google.cloud import storage, aiplatform

# Project configuration
PROJECT_ID = "astral-net-472017-i3"
REGION = "asia-south1"
BUCKET_NAME = "hackathon1512"
MODEL_NAME = "deepfake-detector"
ENDPOINT_NAME = "truthguard-endpoint"

# Set environment variables
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# Enable GCP APIs
!gcloud config set project {PROJECT_ID}
!gcloud services enable compute.googleapis.com storage.googleapis.com aiplatform.googleapis.com cloudfunctions.googleapis.com

# ============================================
# CELL 4: Load and Test Deepfake Model
# ============================================
from transformers import AutoImageProcessor, AutoModelForImageClassification, pipeline

print("Loading deepfake detection model...")
processor = AutoImageProcessor.from_pretrained("prithivMLmods/Deep-Fake-Detector-v2-Model")
model = AutoModelForImageClassification.from_pretrained("prithivMLmods/Deep-Fake-Detector-v2-Model")

# Create pipeline
pipe = pipeline("image-classification", model=model, image_processor=processor)

# Test the model
print("Testing model...")
test_result = pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/parrots.png")
print(f"Test result: {test_result}")

# ============================================
# CELL 5: Save Model Locally and Upload to GCS
# ============================================
import shutil
import json
from google.cloud import storage

# Save locally
model_dir = "truthguard_deepfake_model"
model.save_pretrained(model_dir)
processor.save_pretrained(model_dir)

# Add metadata
metadata = {
    "model_name": "TruthGuard Deepfake Detector",
    "model_version": "1.0",
    "framework": "transformers",
    "input_type": "image",
    "output_type": "classification",
    "labels": ["Deepfake", "Real"],
    "created_at": "2025-01-15",
    "description": "Deep learning model for detecting deepfake images and videos"
}
with open(f"{model_dir}/metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

# Upload model to GCS
client = storage.Client(project=PROJECT_ID)
try:
    bucket = client.get_bucket(BUCKET_NAME)
except:
    bucket = client.create_bucket(BUCKET_NAME, location=REGION)

for root, dirs, files in os.walk(model_dir):
    for file in files:
        local_path = os.path.join(root, file)
        blob_path = f"models/{model_dir}/{file}"
        blob = bucket.blob(blob_path)
        blob.upload_from_filename(local_path)
        print(f"Uploaded {local_path} to gs://{BUCKET_NAME}/{blob_path}")

# ============================================
# CELL 6: Create Prediction Handler
# ============================================
%%writefile prediction_handler.py
import os
import json
import base64
import io
import logging
from typing import Dict, Any
from PIL import Image
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification, pipeline

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class TruthGuardPredictor:
    def __init__(self, model_dir: str = "truthguard_deepfake_model"):
        self.model_dir = model_dir
        self.model = None
        self.processor = None
        self.pipeline = None
        self.load_model()

    def load_model(self):
        try:
            logger.info(f"Loading model from {self.model_dir}")
            self.processor = AutoImageProcessor.from_pretrained(self.model_dir)
            self.model = AutoModelForImageClassification.from_pretrained(self.model_dir)
            self.pipeline = pipeline("image-classification", model=self.model, image_processor=self.processor)
            logger.info("Model loaded successfully!")
        except Exception as e:
            logger.error(f"Error loading model: {str(e)}")
            raise

    def preprocess_image(self, image_data: str) -> Image.Image:
        try:
            if image_data.startswith('data:image'):
                image_data = image_data.split(',')[1]
            image_bytes = base64.b64decode(image_data)
            image = Image.open(io.BytesIO(image_bytes))
            if image.mode != 'RGB':
                image = image.convert('RGB')
            return image
        except Exception as e:
            logger.error(f"Error preprocessing image: {str(e)}")
            raise

    def predict_image(self, image_data: str) -> Dict[str, Any]:
        image = self.preprocess_image(image_data)
        results = self.pipeline(image)
        deepfake_score = 0.0
        real_score = 0.0
        for result in results:
            if result['label'].lower() == 'deepfake':
                deepfake_score = result['score']
            else:
                real_score = result['score']
        is_deepfake = deepfake_score > real_score
        confidence = max(deepfake_score, real_score)
        return {
            'is_deepfake': is_deepfake,
            'deepfake_probability': float(deepfake_score),
            'real_probability': float(real_score),
            'confidence': float(confidence),
            'risk_level': self._get_risk_level(deepfake_score),
            'model_version': '1.0'
        }

    def _get_risk_level(self, deepfake_score: float) -> str:
        if deepfake_score > 0.8:
            return 'very_high'
        elif deepfake_score > 0.6:
            return 'high'
        elif deepfake_score > 0.4:
            return 'medium'
        elif deepfake_score > 0.2:
            return 'low'
        else:
            return 'very_low'

# ============================================
# CELL 7: Create Cloud Function Code
# ============================================
%%writefile cloud_function_main.py
import os
import json
from flask import Flask, request, jsonify
from google.cloud import storage
import functions_framework
from prediction_handler import TruthGuardPredictor

app = Flask(__name__)
predictor = None
BUCKET_NAME = "hackathon1512"
MODEL_DIR = "truthguard_deepfake_model"

def initialize_predictor():
    global predictor
    if predictor is None:
        if not os.path.exists(MODEL_DIR):
            download_model_from_gcs()
        predictor = TruthGuardPredictor(MODEL_DIR)

def download_model_from_gcs():
    client = storage.Client()
    bucket = client.bucket(BUCKET_NAME)
    os.makedirs(MODEL_DIR, exist_ok=True)
    blobs = bucket.list_blobs(prefix=f"models/{MODEL_DIR}/")
    for blob in blobs:
        if not blob.name.endswith('/'):
            local_filename = blob.name.split('/')[-1]
            local_path = os.path.join(MODEL_DIR, local_filename)
            blob.download_to_filename(local_path)

@functions_framework.http
def predict_deepfake(request):
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'POST, OPTIONS',
        'Access-Control-Allow-Headers': 'Content-Type',
    }
    if request.method == 'OPTIONS':
        return ('', 204, headers)
    initialize_predictor()
    request_json = request.get_json()
    if 'image' in request_json:
        result = predictor.predict_image(request_json['image'])
        analysis_type = 'image'
    else:
        return jsonify({'error': 'No image data provided'}), 400
    response_data = {
        'success': True,
        'analysis_type': analysis_type,
        'results': result,
        'model_info': {'name': 'TruthGuard Deepfake Detector', 'version': '1.0', 'framework': 'transformers'}
    }
    return (json.dumps(response_data), 200, headers)

@app.route('/predict', methods=['POST', 'OPTIONS'])
def local_predict():
    return predict_deepfake(request)

if __name__ == '__main__':
    app.run(debug=True, port=8080)

# ============================================
# CELL 8: Create requirements.txt
# ============================================
%%writefile requirements.txt
google-cloud-storage==2.19.0
transformers==4.36.0
torch==2.1.0
torchvision==0.16.0
Pillow==10.1.0
opencv-python-headless==4.8.1.78
numpy==1.24.3
flask==3.0.0

# ============================================
# CELL 9: Frontend Configuration
# ============================================
%%writefile config.json
{
  "gcp": {
    "project_id": "astral-net-472017-i3",
    "region": "asia-south1",
    "bucket_name": "hackathon1512",
    "function_url": "YOUR_FUNCTION_URL_HERE",
    "model_name": "truthguard-deepfake-detector"
  },
  "api": {
    "endpoints": {
      "deepfake_detection": "/predict"
    }
  },
  "limits": {
    "max_file_size": 16777216,
    "supported_formats": {
      "images": ["jpg", "jpeg", "png", "gif", "bmp", "webp"]
    }
  }
}

print("✅ Setup complete! Update FUNCTION_URL in config.json and deploy your Cloud Function.")


SyntaxError: invalid syntax (ipython-input-1822452552.py, line 245)